In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds

import warnings

# Ignore warnings related to corrupt JPEG data
warnings.filterwarnings('ignore', message='Corrupt JPEG data')
# Ignore unknown JFIF warnings
warnings.filterwarnings('ignore', message='unknown JFIF revision number')

In [6]:
# Load the Cats and Dogs dataset from TensorFlow Datasets
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
train_dataset= dataset['train']

In [7]:
def format_image(image, label):
    try:
        image = tf.image.resize(image, (150, 150))  # Resize the image
        image = image / 255.0  # Normalize the image
    except tf.errors.InvalidArgumentError:
        return None, None
    return image, label

In [8]:

train_dataset = train_dataset.map(format_image)
train_dataset = train_dataset.filter(lambda x, y: x is not None)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * info.splits['train'].num_examples)
train_images = train_dataset.take(train_size)
validation_images = train_dataset.skip(train_size)

# Batch the data and set up for prefetching to improve performance
BATCH_SIZE = 32
train_images = train_images.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_images = validation_images.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [9]:
# Load the pretrained VGG16 model without the top layer (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model so its weights are not updated during training
base_model.trainable = False

In [10]:
# Add new layers on top of the pretrained model for binary classification
model = models.Sequential([
    base_model,  # Pretrained VGG16 model
    layers.Flatten(),  # Flatten the 3D output from the convolutional layers
    layers.Dense(256, activation='relu'),  # Fully connected layer with 256 units
    layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification (Cats vs. Dogs)
])

In [11]:
# Compile the model with Adam optimizer and binary crossentropy loss function
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model using the augmented data from the ImageDataGenerator
history = model.fit(train_images, epochs=10, validation_data=validation_images)


Epoch 1/10


I0000 00:00:1726410577.618947   58199 service.cc:146] XLA service 0x7f5580086600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726410577.618973   58199 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-09-15 16:29:37.635340: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-15 16:29:37.755496: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


      3/Unknown 6s 41ms/step - accuracy: 0.4670 - loss: 0.9408

I0000 00:00:1726410582.670505   58199 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    225/Unknown 15s 40ms/step - accuracy: 0.7446 - loss: 0.5079

...

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9620 - loss: 0.1010

2024-09-15 16:34:47.327061: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5182763942082654173
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 53ms/step - accuracy: 0.9620 - loss: 0.1010 - val_accuracy: 0.9203 - val_loss: 0.1962


2024-09-15 16:34:54.188852: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4040984120745805549
2024-09-15 16:34:54.188894: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5182763942082654173


In [13]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_images)
print(f"Validation accuracy with transfer learning: {val_acc}")

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


      3/Unknown 1s 42ms/step - accuracy: 0.9618 - loss: 0.1055 

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


     55/Unknown 3s 43ms/step - accuracy: 0.9267 - loss: 0.1732

Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


...

Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9


    131/Unknown 6s 43ms/step - accuracy: 0.9234 - loss: 0.1864

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


146/146 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - accuracy: 0.9229 - loss: 0.1877
Validation accuracy with transfer learning: 0.9202665090560913


2024-09-15 16:35:01.279476: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4040984120745805549
2024-09-15 16:35:01.279518: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 5182763942082654173
